In [88]:
%load_ext autoreload
%autoreload 2

from modules.negative_set import generate_negative_set, get_box_parameters
from modules import data, models, descriptor_vector, validation, selection
from modules.selection import try_classifiers, try_params
import numpy as np
from skimage.transform import resize
import pickle
from pprint import pprint as pp

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
# Store
store = []

In [14]:
# Load data
#full_images = data.load_images()
#full_labels = data.load_labels()

In [89]:
# Global Params
OFFSET = 0
LIMIT = 100
NEG_SIZE = 150
TRAIN_RATE = 0.75

only_one_training = True
limit_score = 0.5
slide_step = (60,50)
downscale_step = 30
box_size = (150, 60)

# Model
CLASSIFIER = 'random_forest'
MODEL_PARAMS = {
    'n_estimators': 50,
    'class_weight': { -1: 1, 1: 1.5}
}
VECTORIZATION_PARAMS = {
    'vectorize': descriptor_vector.hog,
    'vectorize_args': [6],
} 

In [77]:
# Limit if needed
if LIMIT:
    images = full_images[OFFSET:OFFSET+LIMIT]
    labels = full_labels[full_labels[:,0] <= OFFSET+LIMIT]
    labels = labels[labels[:,0] > OFFSET]
else:
    images = full_images
    labels = full_labels
print(f"Using {len(images)}/{len(full_images)} images & {len(labels)}/{len(full_labels)} labels")

Using 100/1000 images & 134/1284 labels


In [78]:
# Label sets
print("Generating negative set...")
negatives = generate_negative_set(images, labels, set_size=NEG_SIZE)

print("Creating train & validation sets with negatives...")
all_labels = np.concatenate([labels, negatives])
train_labels, valid_labels = label_sets = data.train_valid_sets(len(images), all_labels, TRAIN_RATE)
print(f"Train: {len(np.unique(train_labels[:,0]))}, Valid: {len(np.unique(valid_labels[:,0]))}")

Generating negative set...
Creating train & validation sets with negatives...
Train: 60, Valid: 40


In [64]:
# ( 1 ) --- Singe --- Train, predict, validate
clf = models.create_model(CLASSIFIER, MODEL_PARAMS)
kwargs = { 'limit_score': limit_score, 'slide_step': slide_step, 'downscale_step': downscale_step }

print("Training...")
train_results = models.train(clf, images, box_size, train_labels, **VECTORIZATION_PARAMS,
                             only_one_training=only_one_training, **kwargs)

print("Predicting with windows...")
valid_indexes = np.unique(valid_labels[:,0]) - 1
predictions = models.predict(clf, images, box_size, **VECTORIZATION_PARAMS, only=valid_indexes, **kwargs)

results = validation.rate_predictions(predictions, valid_labels)
print("Done")

print("\n------\n")
if results not in store: store.append(results)
pp(results)

Training...
First training...
Predicting with windows...


Predicting windows: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:12<00:00,  8.26it/s]


Done

------

{'false_pos': 9,
 'missing_predictions': 8,
 'no_faces': 35,
 'no_images': 25,
 'no_predictions': 42,
 'true_pos': 33}


In [ ]:
# ( 2 ) --- Multiple --- Try multiple params
clf_name = 'random_forest'
global_params = {
    # General
    'box_size': (150, 60),

    'n_estimators': 50,
    'class_weight': { -1: 1, 1: 1.2},

    'vectorization_params': {
        'vectorize': descriptor_vector.hog,
        'vectorize_args': [6],
    },
}

kwargs = {
    'limit_score': 0.5,
    'slide_step': (60,50),
    'downscale_step': 30,
    'only_one_training': True,
}

changing_params = {
    'box_size': [
        (80, 60),
        (90, 60),
        (100, 60),
        (110, 70),
        (120, 70),
        (130, 70),
        (110, 80),
        (120, 80),
        (130, 80),
        (120, 90),
        (130, 90),
        (140, 90),
        (150, 90),
        (150, 60),
    ]
}

results = try_params(images, label_sets, clf_name, global_params, changing_params, **kwargs)
print("\n------\n")
if results not in store: store.append(results)
pp(results)

## Trying parameter `box_size`...
### with value `(80, 60)`
First training...



Predicting windows:  65%|████████████████████████████████████████████████████████████████████████████████████▌                                             | 65/100 [00:20<00:17,  2.05it/s]